<a id="5"></a>
<font color="blue" size=+2.5><b>2.2 Library Import</b></font>

In [ ]:
# !pip install fastai
# import libraries
import missingno as msno
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

<a id="6.1"></a>
# <font color="blue" size=+2.5><b>Data Loading</b></font>

../input/textdb3/fake_or_real_news.csv

In [ ]:
train = pd.read_csv("/kaggle/input/textdb3/fake_or_real_news.csv")

In [ ]:
train

In [ ]:
train_df = train[["text","label"]]

In [ ]:
train_df = train_df[:1000]

In [ ]:
train_df

In [ ]:
data_lm = (TextList.from_df(train_df)
           #Inputs: all the text files in path
            .split_by_rand_pct(0.20)
           #We randomly split and keep 20% for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=128))
data_lm.save('tmp_lm')

In [ ]:
data_lm.show_batch()

Now we can create a language model based on the architecture 
[AWD_LSTM](https://docs.fast.ai/text.models.html#AWD_LSTM)

In [ ]:
# Language model AWD_LSTM
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

<a id="9"></a>
<font color="blue" size=+2.5><b> 3.4 Model Summary </b></font>


In [ ]:
print('Model Summary:')
print(learn.layer_groups)

<a id="10"></a>
<font color="blue" size=+2.5><b> 3.5 Finding LR </b></font>


**Lets train our language model. First, we call lr_find to analyze and find an optimal learning rate for our problem, then we fit or train the model for a few epochs. Finally we unfreeze the model and runs it for a few more epochs. So we have a encoder trained and ready to be used for our classifier and it is recorded on disk.**

<a id="11"></a>
<font color="blue" size=+2.5><b> 3.6 HyperParameter Tuning For Model Training </b></font>

In [ ]:
learn.fit_one_cycle(1, 1e-2)
learn.save('lm_hyper')

<a id="12"></a>
<font color="blue" size=+2.5><b> 3.7 Saving Model After Training </b></font>

In [ ]:
learn.save_encoder('ft_enc')


<a id="13"></a>
<font color="blue" size=+2.5><b> 4. Building and Training a Text Classifier </b></font>

<a id="14"></a>
<font color="blue" size=+2.5><b> 4.1 Loading Data For Text Classification </b></font>


In [ ]:
data_clas = (TextList.from_df(train_df, cols=["text"], vocab=data_lm.vocab)
             .split_by_rand_pct(0.3)
             .label_from_df('label')
             .databunch(bs=32))

data_clas.save('tmp_class')


<a id="15"></a>
<font color="blue" size=+2.5><b> 4.2 Loading Text Classification Model </b></font>


In [ ]:
help(learn)

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)


**we will have to load the encoder previously trained (the language model).**

In [ ]:
learn.load_encoder('ft_enc')

**Now, the training cycle is repeated: lr_find, freeze except last layer,..., unfreeze the model and saving the final trained model.**

In [ ]:
learn.freeze_to(-1)
learn.summary()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
help(learn)

In [ ]:
learn.fit_one_cycle(1, 1e-1)

In [ ]:
learn.save('stage1')

<a id="22"></a>
<font color="blue" size=+2.5><b>5.1 Interpret the results</b></font>
<br/>

In [ ]:
from fastai.vision import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(6,6), dpi=60)

In [ ]:
interp = TextClassificationInterpretation.from_learner(learn)
interp.show_top_losses(10)

<a id="23"></a>
<font color="blue" size=+2.5><b>5.3 Save and Load Model</b></font>
<br/>


In [ ]:
learn.export()
learn.model_dir = "/kaggle/working"
learn.save("stage-1",return_path=True)

<a id="24"></a>
<font color="blue" size=+2.5><b>5.4 Sources</b></font>
<br/>
* [Fastai MOOC](https://course.fast.ai/)
* [Fastai library](https://docs.fast.ai/)

<a id="18"></a>
<font color="blue" size=+2.5><b>Feedback and Support</b></font>
<br/>
* Your feedback is much appreciated
* Please UPVOTE if you LIKE this notebook
* Comment if you have any doubts or you found any errors in the notebook